In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [5]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for the QeCalculator class

This tutorial describes the usage of the QeCalculator class, that manages the run of a calculation with the QuantumESPRESSO package.

In [2]:
run_dir = 'QeCalculator_test_new'

## Perform a scf computations for silicon

We init the PwInput object using an exsisting input file. Then we define an inputs with the associated name
using the energy_cutoff as a parameter

In [16]:
energy_cutoffs = 50

In [17]:
inp = I.PwInput(file='IO_files/si_scf.in')
prefix = 'ecut_%s'%energy_cutoffs
inp.set_kpoints(points = [6,6,6])
inp.set_prefix(prefix)
inp.set_energy_cutoff(energy_cutoffs)
name = prefix
inp

{'control': {'verbosity': "'high'",
  'pseudo_dir': "'../pseudos'",
  'calculation': "'scf'",
  'prefix': "'ecut_50'"},
 'system': {'force_symmorphic': '.true.',
  'occupations': "'fixed'",
  'ibrav': '2',
  'celldm(1)': '10.3',
  'ntyp': '1',
  'nat': '2',
  'ecutwfc': 50},
 'electrons': {'conv_thr': '1e-08'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.125, 0.125, 0.125]], ['Si', [-0.125, -0.125, -0.125]]]},
 'kpoints': {'type': 'automatic', 'values': ([6, 6, 6], [0.0, 0.0, 0.0])},
 'cell_parameters': {},
 'file': 'IO_files/si_scf.in'}

Note that we have chosen the value of the prefix of the input object as the name of the file. In this way the inp, log and xml file created by QuantumESPRESSO
have the same name of the prefix folder.

Now we define an intance of the QeCalculator. For this example we use a direct scheduler

In [8]:
C.QeCalculator?

Init signature:
C.QeCalculator(
    omp=1,
    mpi=2,
    mpi_run='mpirun -np',
    executable='pw.x',
    scheduler='direct',
    skip=True,
    clean_restart=True,
    verbose=True,
    **kwargs,
)
Docstring:     
Perform a QuantumESPRESSO calculation. Computations are managed by a scheduler that,
in the actual implementation of the class, can be `direct` or `slurm`.

Parameters:
   omp (:py:class:`int`) : value of the OMP_NUM_THREADS variable
   mpi (:py:class:`int`) : number of mpi processes
   mpi_run (:py:class:`string`) : command for the execution of mpirun, e.g. 'mpirun -np' or 'mpiexec -np'
   executable (:py:class:`string`) : set the executable (pw.x, ph.x, ..) of the QuantumESPRESSO package
   scheduler (:py:class:`string`) : choose the scheduler used to submit the job, actually the choices implemented are
        'direct' that runs the computation using the python subprocess package and 'slurm' that creates a slurm script
   skip (:py:class:`bool`) : if True evaluate if the

In [9]:
code = C.QeCalculator(mpi=2)
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


{'omp': 1,
 'mpi': 2,
 'mpi_run': 'mpirun -np',
 'executable': 'pw.x',
 'scheduler': 'direct',
 'skip': True,
 'clean_restart': True,
 'verbose': True}

We run the computation passing the the input object and the associated name to the run method of the 
calculator.

The effects of various options of the calculator like, skip, clean_restart, dry_run, and wait_end_run can be checked by adding
these options in the run methods. For instance

In [10]:
result = code.run(run_dir=run_dir,input=inp,name=name,other_variable = 1,skip=False, dry_run = True)
result

Dry_run option active. Computation not performed
The wait_end_run is False or the dry_run option is active. The calculator proceedes to the postprocessing
Expected file QeCalculator_test_new/ecut_50.save/data-file-schema.xml not Found
Check if wait_end_run is False or the dry_run option is active. Otherwise a possible error has occured during the computation


'QeCalculator_test_new/ecut_50.save/data-file-schema.xml'

We perform a simple called to run that perform the computation

In [28]:
result = code.run(run_dir=run_dir,input=inp,name=name,other_variable = 1)
result

run command: cd QeCalculator_test_new; mpirun -np 2 pw.x -inp ecut_50.in > ecut_50.log
computation still running...
computation ended


'QeCalculator_test_new/ecut_50.save/data-file-schema.xml'

We observe that the output of the run method is a string with the the data-file-schema.xml

After the run all the parameters passed to the calculator are written in the run_options attribute

In [12]:
code.run_options

{'omp': 1,
 'mpi': 2,
 'mpi_run': 'mpirun -np',
 'executable': 'pw.x',
 'scheduler': 'direct',
 'skip': True,
 'clean_restart': True,
 'verbose': True,
 'run_dir': 'QeCalculator_test_new',
 'input': {'control': {'verbosity': "'high'",
   'pseudo_dir': "'../pseudos'",
   'calculation': "'scf'",
   'prefix': "'ecut_50'"},
  'system': {'force_symmorphic': '.true.',
   'occupations': "'fixed'",
   'ibrav': '2',
   'celldm(1)': '10.3',
   'ntyp': '1',
   'nat': '2',
   'ecutwfc': 50},
  'electrons': {'conv_thr': '1e-08'},
  'ions': {},
  'cell': {},
  'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
  'atomic_positions': {'type': 'crystal',
   'values': [['Si', [0.125, 0.125, 0.125]],
    ['Si', [-0.125, -0.125, -0.125]]]},
  'kpoints': {'type': 'automatic', 'values': ([6, 6, 6], [0.0, 0.0, 0.0])},
  'cell_parameters': {},
  'file': 'IO_files/si_scf.in'},
 'name': 'ecut_50',
 'other_variable': 1}

Instead, let see what happens if the simulation fails. For instance if we provide an empty input to code

In [19]:
inp2 = I.PwInput()

In [23]:
pref2 = 'si_scf_test2'
inp2.set_prefix(pref2)
inp2

{'control': {'prefix': "'si_scf_test2'"},
 'system': {},
 'electrons': {},
 'ions': {},
 'cell': {},
 'atomic_species': {},
 'atomic_positions': {},
 'kpoints': {},
 'cell_parameters': {}}

In [24]:
result2 = code.run(input = inp2, run_dir = run_dir,name=pref2) 
result2

run command: cd QeCalculator_test_new; mpirun -np 2 pw.x -inp si_scf_test2.in > si_scf_test2.log
computation still running...
computation ended
Expected file QeCalculator_test_new/si_scf_test2.save/data-file-schema.xml not Found
Check if wait_end_run is False or the dry_run option is active. Otherwise a possible error has occured during the computation


'QeCalculator_test_new/si_scf_test2.save/data-file-schema.xml'

In this case the calculator provide a warning since the data-file-schema has not been found after the computation

### Usage of the skip parameter

If we repeat a calculation that has been already performed and skip = True the class skip its computation, for instance

In [26]:
result = code.run(run_dir=run_dir,input=inp,name=name, skip = True)

Skip the run of ecut_50


### Test of the slurm scheduler

If the `slurm` scheduler is chosen the calculator prepare the slurm script and submit it. 

In [27]:
result = code.run(run_dir=run_dir,input=inp,name=name, scheduler = 'slurm', skip = False, dry_run = True)

delete log file: QeCalculator_test_new/ecut_50.log
delete xml file: QeCalculator_test_new/ecut_50.xml
delete folder: QeCalculator_test_new/ecut_50.save
run command: mpirun -np 2 pw.x -inp ecut_50.in > ecut_50.log
Dry_run option active. Script not submitted
The wait_end_run is False or the dry_run option is active. The calculator proceedes to the postprocessing
Expected file QeCalculator_test_new/ecut_50.save/data-file-schema.xml not Found
Check if wait_end_run is False or the dry_run option is active. Otherwise a possible error has occured during the computation


The slurm script is written in the run_dir. The execution of the run requires that the slurm scheduler is installed. However with the
dry_run option we can write the script on disk.

## Perform a nscf computation for silicon. Usage of the source_dir option

We show how to perform a pw nscf calculation using the results of the first scf run as an input.

Before running this computation ensure that the scf one has been computed and that the save folder is written in the run_dir.

In [29]:
num_bands = 8

In [30]:
inp.set_nscf(num_bands,force_symmorphic=True)
prefix = 'bands_%s'%num_bands
inp.set_prefix(prefix)
inp.set_energy_cutoff(50)
name = prefix

In [32]:
import os
result = code.run(input=inp,run_dir=run_dir,name=name,source_dir=os.path.join(run_dir,'ecut_50.save'))
result

Copy source_dir QeCalculator_test_new/ecut_50.save in the QeCalculator_test_new/bands_8.save
run command: cd QeCalculator_test_new; mpirun -np 2 pw.x -inp bands_8.in > bands_8.log
computation still running...
computation ended


'QeCalculator_test_new/bands_8.save/data-file-schema.xml'

Instead, if skip = False the class delete the existing output files before running the computation again. 

In [33]:
result = code.run(input=inp,run_dir=run_dir,name=name,source_dir=os.path.join(run_dir,'ecut_50.save'),skip=False)
result

delete log file: QeCalculator_test_new/bands_8.log
delete xml file: QeCalculator_test_new/bands_8.xml
delete folder: QeCalculator_test_new/bands_8.save
Copy source_dir QeCalculator_test_new/ecut_50.save in the QeCalculator_test_new/bands_8.save
run command: cd QeCalculator_test_new; mpirun -np 2 pw.x -inp bands_8.in > bands_8.log
computation still running...
computation ended


'QeCalculator_test_new/bands_8.save/data-file-schema.xml'